# AGREGACION DE DATOS DESDE EL DATASET DEL PROGRAMA DE INDICADORES DE GESTIÓN DE ORGANISMOS OPERADORES

**Descripcion**

En este documento se describen las consideraciones que se tomaron para la agregación de datos desde el dataset proporcionado por el Programa de indicadores de Gestion de Organismos Operadores para su uso en la Plataforma de Conocimiento de Ciudades   Sustentables. 


**Siglas**

- **INEGI**: Instituto Nacional de Geografía y Estadística.
- **PCCS**: Plataforma de Conocimiento de Ciudades Sustentables.
- **SUN**: Sistema Urbano Nacional
- **PIGOO**: Programa de Indicadores del Gestión de Organismos Operadores

**Repositorio de datasets**

Dataset Pigoo: 
https://github.com/INECC-PCCS/01_Dmine/blob/master/Datasets/Pigoo/CiudadesPIGOO_ClaveInegi.xlsx

Dataset Ciudades que forman parte del subsistema principal del SUN: 
https://github.com/INECC-PCCS/01_Dmine/blob/master/00_Generales/sun_main.csv

In [1]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib

module_path = r'D:\PCCS\01_Dmine\Scripts'
if module_path not in sys.path:
    sys.path.append(module_path)
from SUN.asignar_sun import asignar_sun
from SUN_integridad.SUN_integridad import SUN_integridad
from SUN.CargaSunPrincipal import getsun

In [5]:
# Configuracion del sistema
import sys; print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


## 1. Revisión y estandarización inicial al DataSet Pigoo

El 2 de octubre de 2017 se realizó por medio de correo electrónico, una solicitud de aclaracion al PIGOO del nivel de desagregación de los datos disponibles en su página (Ver archivo Aclaracion_desagregacion.pdf).

En respuesta a esta solicitud, el PIGOO proporcionó un archivo de excel (**CiudadesPIGOO_ClaveInegi.xlsx**) que incluye una clasificación de las ciudades del dataset PIGOO por clave geoestadística

In [5]:
# Carga del dataset "CiudadesPIGOO_ClaveInegi.xlsx" al sistema
pigoo_inegi = r'D:\PCCS\01_Dmine\Datasets\Pigoo\CiudadesPIGOO_ClaveInegi.xlsx'
pigoo_inegi_df = pd.read_excel(pigoo_inegi, sheetname='OOAPAS-PIGOO', index_col=0,
                      dtype={'Clave-Estado-Inegi': str,
                             'Clave-Municipio-Inegi': str,
                             'Clave-Localidad-Inegi': str})
pigoo_inegi_df.head()

,Nombre- PIGOO,Organismo Operador,Siglas,Estado,Municipio-Inegi,Loc-Inegi,Clave-Estado-Inegi,Clave-Municipio-Inegi,Clave-Localidad-Inegi
N°,,,,,,,,,
1,Aguascalientes,Comisión Ciudadana de Agua Potable y Alcantari...,CCAPAMA,Aguascalientes,Aguascalientes,NaN,01,001,nan
2,Jesús María,Comisión de Agua Potable Alcantarillado y Sane...,CAPAS,Aguascalientes,Jesús María,NaN,01,005,nan
3,Calvillo,Organismo Operador de Servicios de Agua de Cal...,OOSAC,Aguascalientes,Calvillo,NaN,01,003,nan
4,Asientos,Sistema de Agua Potable y Alcantarillado de As...,SAPA,Aguascalientes,Asientos,NaN,01,002,nan
5,Tecate,Comisión Estatal de Servicios Públicos de Teca...,CESPTE,Baja California,Tecate,NaN,02,003,nan


Gracias a que este dataset ya contiene etiquetas con claves geoestadísticas, es posible clasificarlas a su vez dentro de acuerdo con el Sistema Urbano Nacional. Para hacer la clasificación, se utiliza un algoritmo elaborado previamente que identifica la clave geoestadística municipal de 5 dígitos ("CVE_MUN") en cada renglón y la clasifica el municipio identificado asignándole la clave del Sistema Urbano Nacional ("CVE_SUN") a la que pertenece.
Es importante señalar las limitaciones de este algoritmo:
1. El algoritmo, además de clasificar los municipios, les asigna el nombre estándar como aparece en el marco geoestadístico del INEGI y el nombre de las ciudades como aparece en el Sistema Urbano Nacional.
2. Unicamente clasifica municipios que forma parte del Subsistema Principal del SUN. Desecha aquellos que no forman parte del Subsistema Principal.
3. Unicamente clasifica renglones cuya clave geoestadística municipal es de 5 dígitos ("CVE_MUN"). Esto es relevante pues en el dataset de entrada, la "Ciudad de México"  únicamente cuentan con 2 dígitos de clave geoestadística

In [10]:
# Estandarizacion de clave geoestadistica municipal a 5 dígitos
pigoo_inegi_df['CVE_MUN'] = pigoo_inegi_df['Clave-Estado-Inegi'].map(str) + pigoo_inegi_df['Clave-Municipio-Inegi']
# Clasificación de acuerdo al Sistema Urbano Nacional
variables_SUN = ['CVE_MUN', 'NOM_MUN', 'CVE_SUN', 'NOM_SUN', 'TIPO_SUN', 'NOM_ENT']
pigoo_sun = asignar_sun(pigoo_inegi_df, vars=variables_SUN)
Columnas = ['Nombre- PIGOO', 'CVE_MUN', 'NOM_MUN', 'CVE_SUN', 'NOM_SUN', 'TIPO_SUN', 'NOM_ENT', 'Organismo Operador', 'Siglas', 'Loc-Inegi']
pigoo_sun[Columnas].head()

,Nombre- PIGOO,CVE_MUN,NOM_MUN,CVE_SUN,NOM_SUN,TIPO_SUN,NOM_ENT,Organismo Operador,Siglas,Loc-Inegi
0,Aguascalientes,01001,Aguascalientes,001,Aguascalientes,1,Aguascalientes,Comisión Ciudadana de Agua Potable y Alcantari...,CCAPAMA,NaN
1,Jesús María,01005,Jesús María,001,Aguascalientes,1,Aguascalientes,Comisión de Agua Potable Alcantarillado y Sane...,CAPAS,NaN
2,Tecate,02003,Tecate,002,Tijuana,1,Baja California,Comisión Estatal de Servicios Públicos de Teca...,CESPTE,NaN
3,Tijuana,02004,Tijuana,002,Tijuana,1,Baja California,Comisión Estatal de Servicios Públicos de Tiju...,CESPT,NaN
4,Ensenada,02001,Ensenada,060,Ensenada,2,Baja California,Comisión Estatal de Servicios Públicos de Ense...,CESPE,NaN


## 2. Caso Valle de México

El Valle de México es un caso que requiere consideraciones particulares para su agregación a los datasets que se incluirán en la PCCS. Es necesario hacer una distinción entre la entidad llamada "Ciudad de México" y la ciudad llamada "Valle de México". Las delegaciones que componen la Ciudad De México forman parte de la Ciudad Valle de México, que además está compuesta por municipios del Estado de México, Ciudad de México e Hidalgo.

La Ciudad de México (Antes Distrito Federal) está dividida en 16 delegaciones, cada una identificada por el INEGI con una clave geoestadística. 

In [36]:
sun = getsun().set_index('CVE_SUN')
cdmx = sun[sun['CVE_ENT'] == '09']
datos = ['CVE_ENT','CVE_MUN', 'NOM_MUN']
cdmx[datos]

,CVE_ENT,CVE_MUN,NOM_MUN
CVE_SUN,,,
013,09,09002,Azcapotzalco
013,09,09003,Coyoacán
013,09,09004,Cuajimalpa de Morelos
013,09,09005,Gustavo A. Madero
013,09,09006,Iztacalco
013,09,09007,Iztapalapa
013,09,09008,La Magdalena Contreras
013,09,09009,Milpa Alta
013,09,09010,Álvaro Obregón


En el Dataset de PIGOO la Ciudad de México está considerada como una unidad monolítica, manejada por el Sistema de Aguas de La Ciudad de México.

In [8]:
pigoo_inegi_df.loc[42]

Nombre- PIGOO                                            Ciudad De México
Organismo Operador       Sistema de Aguas de La Ciudad de México (SACMEX)
Siglas                                                               SACM
Estado                                                   Distrito Federal
Municipio-Inegi                                          Ciudad de México
Loc-Inegi                                                             NaN
Clave-Estado-Inegi                                                     09
Clave-Municipio-Inegi                                                 nan
Clave-Localidad-Inegi                                                 nan
CVE_MUN                                                             09nan
Name: 42, dtype: object

En el SUN, el Valle de México se compone de 76 municipios distribuidos en 3 entidades: 16 de la Ciudad de México, 59 del Estado de México y 1 de Hidalgo.

In [37]:
vdmx = sun[sun.index == '013']
vdmx[datos]

,CVE_ENT,CVE_MUN,NOM_MUN
CVE_SUN,,,
013,09,09002,Azcapotzalco
013,09,09003,Coyoacán
013,09,09004,Cuajimalpa de Morelos
013,09,09005,Gustavo A. Madero
013,09,09006,Iztacalco
013,09,09007,Iztapalapa
013,09,09008,La Magdalena Contreras
013,09,09009,Milpa Alta
013,09,09010,Álvaro Obregón


## 3. Consideraciones Particulares para cada indicador generado desde el PIGOO

El dataset PIGOO contiene 8 parámetros relevantes para la PCCS. Para ajustar la agregación de estos parámetros a la requerida por la PCCS, se tomaron en cuenta las siguientes consideraciones:

**P0113 - Consumo (l/h/d)**

Promedio

**P0114 - Dotación (l/h/d)**

Promedio

**P0115 - Macromedición**

Promedio

**P0116 - Pérdidas por longitud de red**

Promedio

**P0117 - Rehabilitación de tubería  (%)**

Promedio

**P0118 - Rehabilitación de tomas domiciliarias  (%)**

Promedio

**P0119 - Rehabilitación de tomas domiciliarias  (%)**

Promedio

**P0120 - Rehabilitación de tomas domiciliarias  (%)**

Promedio

## 4. Estandarizacion del dataset PIGOO

In [7]:
# Descarga del dataset
dirdestino = r'D:\PCCS\00_RawData\01_CSV'
archivodestino = r'pigoo.csv'
destino = r'{}\{}'.format(dirdestino, archivodestino)
fuente = r'http://www.pigoo.gob.mx/dashboard/exportar/excel/exportar_consulta_excel.php?tipo=max'
print('Descargando dataset ... ... ... ...')
urllib.request.urlretrieve(fuente, destino)
print('dataset descargado en {}'.format(destino))
dataset = pd.read_csv(destino,
                        header = 3)
dataset = dataset.set_index('ciudad')
dataset['ciudad'] = dataset.index

Descargando dataset ... ... ... ...
dataset descargado en D:\PCCS\00_RawData\01_CSV\pigoo.csv


Después de la solicitud de aclaración de desagregación que se hizo al pigoo, nos mandaron un dataset en donde nuevamente los nombres de las ciudades vienen cambiados. Se realizó un dataset para definir con certeza las correspondencias entre los nombres de ciudades en el dataset disponible en la pagina de PIGOO, los nombres del archivo de aclaración de desagregación y la Clave Municipal de 5 dígitos del marco geoestadístico de INEGI.

In [27]:
dataset.head()

,indicador,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
ciudad,,,,,,,,,,,,,,,
"Tecate, Baja California",Tomas con servicio continuo (%),91.99,91.46,90.40,90.68,90.14,90.43,89.82,95.96,96.04,96.23,99.39,99.44,99.12,99.23
"Tijuana, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
"Ensenada, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,78.13,98.83,98.77,99.00,97.79,97.71,97.54,97.50,NaN
"Mexicali, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
"Torreón, Coahuila",Tomas con servicio continuo (%),99.69,99.77,99.76,99.71,99.72,99.70,99.71,92.91,100.00,100.00,100.00,99.71,100.00,100.00


In [25]:
nom_estandar = pd.read_excel(r'D:\PCCS\01_Dmine\Datasets\Pigoo\standard-pigoo_ooapas_geo.xlsx')
nom_estandar.head()

,Nombre en CiudadesPIGOO_ClaveInegi,CVE_MUN,Nombre en Dataset PIGOO
0,Tecate,2003,"Tecate, Baja California"
1,Tijuana,2004,"Tijuana, Baja California"
2,Ensenada,2001,"Ensenada, Baja California"
3,Mexicali,2002,"Mexicali, Baja California"
4,Torreón,5035,"Torreón, Coahuila"


Utilizando este dataset, ya es posible asignar claves geoestadísticas municipales a las ciudades del dataset PIGOO 

In [32]:
pd.merge(dataset, nom_estandar[['CVE_MUN', 'Nombre en Dataset PIGOO']], how='left', left_on='ciudad', right_on='Nombre en Dataset PIGOO').head()

,indicador,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,ciudad,CVE_MUN,Nombre en Dataset PIGOO
0,Tomas con servicio continuo (%),91.99,91.46,90.40,90.68,90.14,90.43,89.82,95.96,96.04,96.23,99.39,99.44,99.12,99.23,"Tecate, Baja California",2003,"Tecate, Baja California"
1,Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,"Tijuana, Baja California",2004,"Tijuana, Baja California"
2,Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,78.13,98.83,98.77,99.00,97.79,97.71,97.54,97.50,NaN,"Ensenada, Baja California",2001,"Ensenada, Baja California"
3,Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,"Mexicali, Baja California",2002,"Mexicali, Baja California"
4,Tomas con servicio continuo (%),99.69,99.77,99.76,99.71,99.72,99.70,99.71,92.91,100.00,100.00,100.00,99.71,100.00,100.00,"Torreón, Coahuila",5035,"Torreón, Coahuila"


El dataset Pigoo se separará en datasets diferentes para cada indicador. Los indicadores que incluye el dataset PIGOO son los siguientes:

In [23]:
indicadores = set(dataset['indicador'])
for i in indicadores: print(i)

Consumo (l/h/d)
Eficiencia física 2 (%)
Eficiencia física 1 (%)
Rehabilitación de tubería  (%)
Cobertura de alcantarillado reportada (%)
Dotación (l/h/d)
Eficiencia comercial (%)
Usuarios abastecidos con pipas (%)
Cobertura de agua potable reportada (%)
Eficiencia global (%)
Padrón de usuarios (%)
Macromedición (%)
Redes e instalaciones (%)
Micromedición(%)
Costos entre volumen producido ($/m³)
Relación inversión-pib (%)
Volumen tratado (%)
Empleados dedicados al control de fugas (Trabajadores/fuga)
Usuarios con pago a tiempo (%)
Pérdidas por toma (m³/toma)
Relación de trabajo (%)
Relación costo - tarifa
Pérdidas por longitud de red (m³/km)
Horas con servicio en zonas de tandeo
Rehabilitación de tomas domiciliarias  (%)
Tomas con servicio continuo (%)
Empleados por cada mil tómas (Núm)
Eficiencia de cobro (%)
Reclamaciones por cada mil tomas (Núm)


In [21]:
# Separacion de dataset por indicador
dict = {}
for i in indicadores:
    indicador = dataset[dataset['indicador'] == i]
    dict[i] = indicador

In [22]:
dict

{'Cobertura de agua potable reportada (%)':                                                                         indicador  \
 ciudad                                                                              
 Tecate, Baja California                   Cobertura de agua potable reportada (%)   
 Tijuana, Baja California                  Cobertura de agua potable reportada (%)   
 Ensenada, Baja California                 Cobertura de agua potable reportada (%)   
 Mexicali, Baja California                 Cobertura de agua potable reportada (%)   
 Torreón, Coahuila                         Cobertura de agua potable reportada (%)   
 Saltillo, Coahuila                        Cobertura de agua potable reportada (%)   
 Irapuato, Guanajuato                      Cobertura de agua potable reportada (%)   
 Celaya, Guanajuato                        Cobertura de agua potable reportada (%)   
 Toluca, México                            Cobertura de agua potable reportada (%)   
 Atlacomulc

In [9]:
set(dataset['indicador'])

{'Cobertura de agua potable reportada (%)',
 'Cobertura de alcantarillado reportada (%)',
 'Consumo (l/h/d)',
 'Costos entre volumen producido ($/m³)',
 'Dotación (l/h/d)',
 'Eficiencia comercial (%)',
 'Eficiencia de cobro (%)',
 'Eficiencia física 1 (%)',
 'Eficiencia física 2 (%)',
 'Eficiencia global (%)',
 'Empleados dedicados al control de fugas (Trabajadores/fuga)',
 'Empleados por cada mil tómas (Núm)',
 'Horas con servicio en zonas de tandeo',
 'Macromedición (%)',
 'Micromedición(%)',
 'Padrón de usuarios (%)',
 'Pérdidas por longitud de red (m³/km)',
 'Pérdidas por toma (m³/toma)',
 'Reclamaciones por cada mil tomas (Núm)',
 'Redes e instalaciones (%)',
 'Rehabilitación de tomas domiciliarias  (%)',
 'Rehabilitación de tubería  (%)',
 'Relación costo - tarifa',
 'Relación de trabajo (%)',
 'Relación inversión-pib (%)',
 'Tomas con servicio continuo (%)',
 'Usuarios abastecidos con pipas (%)',
 'Usuarios con pago a tiempo (%)',
 'Volumen tratado (%)'}

In [12]:
dataset.head()

,indicador,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
ciudad,,,,,,,,,,,,,,,
"Tecate, Baja California",Tomas con servicio continuo (%),91.99,91.46,90.40,90.68,90.14,90.43,89.82,95.96,96.04,96.23,99.39,99.44,99.12,99.23
"Tijuana, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
"Ensenada, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,78.13,98.83,98.77,99.00,97.79,97.71,97.54,97.50,NaN
"Mexicali, Baja California",Tomas con servicio continuo (%),100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
"Torreón, Coahuila",Tomas con servicio continuo (%),99.69,99.77,99.76,99.71,99.72,99.70,99.71,92.91,100.00,100.00,100.00,99.71,100.00,100.00
